# 1. Imports of necessary libraries

In [11]:
import keras #High level interface of TensorFlow
from keras.models import Sequential
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.utils.np_utils import to_categorical 
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image
from keras.optimizers import RMSprop
import tensorflowjs as tfjs
import pandas as pd
from sklearn.model_selection import train_test_split

# 2.Data importing& preprocessing

## 2.1 Importing and entry processing

In [2]:
train_set = pd.read_csv("../data/train.csv") # raw data

# DV
Y_train = train_set["label"]
# IV
X_train = train_set.drop(labels = ["label"], axis = 1)
#X_train

## 2.2Normalization to [0-1]  range

In [3]:
X_train = X_train / 255


X_train = X_train.values.reshape(-1,28,28,1)

Y_train = to_categorical(Y_train, num_classes = 10)

## 2.3Data spliting into train & valid set 

In [4]:
X_train, X_valid , Y_train , Y_valid = train_test_split(X_train, Y_train, test_size = 0.2, random_state = 42)

## 2.4 Augmentation

In [5]:
igm =  ImageDataGenerator (
    featurewise_center=False,
    samplewise_center=False,
    rotation_range=15,
    zoom_range = 0.2,
    width_shift_range=0.3,
    height_shift_range=0.2,
    horizontal_flip=False,
    vertical_flip=False)

igm.fit(X_train)

# 3. Building the architecture of Keras model & Compilation

## 3.1 Architecture

In [6]:
model = Sequential(
        [
            Conv2D(32, (3,3),activation = "relu", input_shape = (28,28,1)),
            MaxPool2D(pool_size = (2, 2)),
            Conv2D(32, (3,3),activation = "relu"),
            MaxPool2D(pool_size = (2, 2)),
            Flatten(),
            Dropout(0.25),
            Dense(units = 32, activation = "relu"),
            Dense(units = 10, activation = "sigmoid")  
        ]
)

## 3.2 Compilation

In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

# 4. Training

In [9]:
model.fit_generator(
    igm.flow(X_train, Y_train, batch_size = 16),
    epochs = 10,
    steps_per_epoch = X_train.shape[0] // 16,
    validation_data = (X_valid, Y_valid)
    )

Epoch 1/10
2100/2100 [==============================] - 32s 15ms/step - loss: 1.0585 - acc: 0.6438 - val_loss: 0.2898 - val_acc: 0.9182
Epoch 2/10
2100/2100 [==============================] - 31s 15ms/step - loss: 0.5279 - acc: 0.8354 - val_loss: 0.1697 - val_acc: 0.9487
Epoch 3/10
2100/2100 [==============================] - 32s 15ms/step - loss: 0.4025 - acc: 0.8768 - val_loss: 0.1395 - val_acc: 0.9563
Epoch 4/10
2100/2100 [==============================] - 33s 16ms/step - loss: 0.3327 - acc: 0.8960 - val_loss: 0.0986 - val_acc: 0.9690
Epoch 5/10
2100/2100 [==============================] - 32s 15ms/step - loss: 0.2908 - acc: 0.9093 - val_loss: 0.0791 - val_acc: 0.9758
Epoch 6/10
2100/2100 [==============================] - 33s 16ms/step - loss: 0.2630 - acc: 0.9177 - val_loss: 0.0794 - val_acc: 0.9757
Epoch 7/10
2100/2100 [==============================] - 33s 15ms/step - loss: 0.2480 - acc: 0.9210 - val_loss: 0.1184 - val_acc: 0.9650
Epoch 8/10
2100/2100 [==========================

# Saving a model

In [12]:
tfjs.converters.save_keras_model(model, './ModelJS')